# Generative AI Media Entertainment Workshop
## Prompt Engineering with Amazon Bedrock

> *This notebook should work well with the **`Python 3`** kernel in SageMaker Studio*

---

In this demo notebook, we demonstrate how to use the [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundation Models.

---

## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [1]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 63.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 119.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:

---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

#### Use different clients
The boto3 provides different clients for Amazon Bedrock to perform different actions. The actions for [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) and [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) are supported by Amazon Bedrock Runtime where as other operations, such as [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html), are handled via [Amazon Bedrock client](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html).


#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.



In [2]:
import json
import os
import sys

import boto3

boto3_bedrock = boto3.client('bedrock')

#### Validate the connection

We can check the client works by trying out the `list_foundation_models()` method, which will tell us all the models available for us to use.

In [3]:
boto3_bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '62c74c94-bd70-4613-9906-bfc4a58a80f0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 04 Jun 2024 00:54:49 GMT',
   'content-type': 'application/json',
   'content-length': '23453',
   'connection': 'keep-alive',
   'x-amzn-requestid': '62c74c94-bd70-4613-9906-bfc4a58a80f0'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-g1-text-02',
   'modelId': 'amazon.titan-embed-g1-text-02',
   'modelName': 'Titan Text Embeddings v2',
   'providerName': 'Amazon',
   'inp

---

## Lab 1 - Text Summarization in Bedrock

Picture yourself at a movie studio, or maybe a video streaming company. You get synopses all the time, and hardly have time to read them. In this Lab you will use Foundation Models (FMs) in Amazon Bedrock to summarize synopsis, so it is much easier to read and digest.

Here is a [Synopsis for the movie Whiplash](https://www.scriptreaderpro.com/wp-content/uploads/2019/07/Film-Synopsis-Example-Whiplash.pdf). We will use it below.



In [4]:
import boto3
import botocore
import json 

bedrock_runtime = boto3.client('bedrock-runtime')

### Cohere Command

Let's create a prompt and ask the model to summarize some text for us.

In [5]:
prompt_data = """
Andrew Neiman, a young jazz student at the Shaffer Conservatory in New York, has one dream:
to go down in history as one of the world’s best drummers. He’s therefore thrilled when Terence
Fletcher, a famous conductor, invites him to join the conservatory’s Studio Band as a core
alternate drummer. Fletcher, however, turns out to be anything but an ordinary teacher. He’s a
sadistic tyrant and Andrew realizes just how much of one when he has a chair hurled at him for
failing to keep time.
At a jazz competition, Andrew misplaces the sheet music to “Whiplash,” meaning their core
drummer can’t play. Andrew, however, can—from memory—and after a first class performance,
Fletcher promotes him to core drummer. But Andrew’s joy won’t last long… In a typically
twisted move, Fletcher bumps Andrew back down to alternate drummer, putting a much lesstalented 
musician in his place. More determined than ever, Andrew breaks up with his girlfriend
and practices until his hands bleed. It pays off… After a grueling five-hour audition, during
which Fletcher kicks furniture and screams at him, Andrew earns back the core spot.
Andrew arrives late for another competition after his bus breaks down, hires a car, then realizes
he left his drumsticks at the car rental office. He races back, retrieves them, but on his way to
the theater, his car is broadsided by a semi. He crawls from the wreckage and runs the rest of
the way, finally arriving on stage bloody and injured. When he struggles to play, Fletcher cooly
dismisses him. Enraged, Andrew attacks Fletcher in front of the audience, which gets him
dismissed from the school.
Andrew files an ethics complaint against Shaffer Conservatory and learns that one of Fletcher’s
former students hanged himself due to his emotional and physical abuse. Andrew agrees to
testify as an anonymous witness and Fletcher is fired. Andrew gives up drumming and, months
later, stumbles upon Fletcher playing piano in a jazz club. They go for a drink, during which
Fletcher explains why he pushed his students so hard: so that they might become the next
Charlie Parker. In Fletcher’s eyes the greats like Parker wouldn’t be discouraged by anything.
He then invites Andrew to drum with his band at a jazz festival. Has Fletcher changed? Andrew
thinks so, and accepts.
On stage at the festival, Fletcher has two surprises for Andrew. One: he knows he testified
against him, and two: they’re starting with a piece Andrew doesn’t know and for which there’s
no sheet music. Unable to play, Andrew leaves the stage humiliated. But he returns, interrupts
Fletcher and cues the band, before launching into a breathtaking solo. Fletcher is taken aback,
but in that moment realizes the enormity of Andrew’s talent and begins to guide him. As
Andrew ends his solo, they share a smile and Fletcher cues the finale.

{INPUT}
"""

Next, we will construct the body and response to pass the prompt above to the Cohere Command text model.

In [6]:
try:

    input_text = prompt_data.replace("{INPUT}", "Summarize the text above:")
    body = json.dumps({
      "prompt": input_text,
      "max_tokens":400,
      "temperature":0.75 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

You don't have access to the model with the specified model ID.                
To troubeshoot this issue please refer to the following resources.                 
https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html                 
https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html



## Refine the output

This is good, but there are ways to refine this result.

## Prompt engineering
Prompt engineering is a discipline focused on developing optimized prompts to efficiently apply language models to various tasks.

Try the same synopsis, but this time followed by "Summarize the text above in one sentence:".

In [7]:
try:
    input_text = prompt_data.replace("{INPUT}", "Summarize the text above in one sentence:")
    
    body = json.dumps({
      "prompt": input_text,
      "max_tokens":400,
      "temperature":0.75 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']
    parse_text

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

You don't have access to the model with the specified model ID.                
To troubeshoot this issue please refer to the following resources.                 
https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html                 
https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html



## Your turn 

Modify the prompt below to output the summary into one paragraph with output similar to the one below:

```
Andrew Neiman is a young drummer who dreams of greatness. He joins the Shaffer Conservatory in New York and is invited to join a band led by the conductor Terence Fletcher. However, Fletcher turns out to be a tyrannical and sadistic bandleader who abuses his students. Despite this, Andrew is determined to succeed and works hard to impress Fletcher. He is eventually promoted to core drummer but is later demoted. This motivates Andrew to practice even harder, and he eventually earns back his core spot. However, Fletcher continues to abuse him, and Andrew eventually testifies against him, leading to Fletcher's dismissal from the school. Months later, Andrew encounters Fletcher again and decides to give him a chance, but Fletcher retaliates by setting up a situation where Andrew fails again. However, Andrew manages to turn the situation around and impresses Fletcher in the end.
```

In [8]:
try:

    USER_INPUT = "" # INPUT YOUR ANSWER HERE
    assert USER_INPUT, "Parameter cannot be an empty string..."

    input_text = prompt_data.replace("{INPUT}", USER_INPUT)


    body = json.dumps({
      "prompt": input_text,
      "max_tokens":400,
      "temperature":0.75 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']
    parse_text

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

AssertionError: Parameter cannot be an empty string...

## Changing prompt parameters.
Lets see what happens if we change the prompt parameters.

Below we are going to change the max_tokens to 50 lets see what happens.


In [ ]:
try:

    body = json.dumps({
      "prompt": input_text,
      "max_tokens":50,
      "temperature":0.2 
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="cohere.command-text-v14", 
      accept= "*/*", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    parse_text = response_body['generations'][0]['text']
    parse_text

    print(parse_text)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

----
# Lab 2


# Text Generation in Bedrock
If you are working with a script (whether for movies, television, game, etc), Foundation Models (FMs) can assist in a number of ways.

## Use FM to create dialogue
**FMs can generate dialogues for you.** - give the model a list of characters and a brief description of the scene, and let FMs generate the dialogues for a Comedy.

In [ ]:
prompt_data = """
Human: as an expert script writer, write the dialogue between a husband and wife for a comedy scene.

Assistant:
"""

In [ ]:
from io import StringIO
import sys
import textwrap

try:

    body = json.dumps({
      "prompt": prompt_data,
      "max_tokens_to_sample":256,
      "top_k":250,
      "stop_sequences":[], #define phrases that signal the model to conclude text generation.
      "temperature":0.5, #Temperature controls randomness; higher values increase diversity, lower values boost predictability.
      "top_p":1 # Top P is a text generation technique, sampling from the most probable tokens in a distribution.
    })

    response = bedrock_runtime.invoke_model(
      body=body,
      modelId="anthropic.claude-v2", 
      accept="application/json", 
      contentType="application/json"
    )


    def llm_output_parser(*args, width: int = 100, **kwargs):
        buffer = StringIO()
        try:
            # Redirect sys.stdout to capture the output
            _stdout = sys.stdout
            sys.stdout = buffer
            print(*args, **kwargs)
            output = buffer.getvalue()
        except Exception as e:
            # Handle any exceptions that may occur during capturing
            print(f"Error capturing output: {e}")
            return
        finally:
            # Restore the original sys.stdout
            sys.stdout = _stdout
    
        try:
            # Wrap lines and print the parsed output
            for line in output.splitlines():
                print("\n".join(textwrap.wrap(line, width=width)))
        except Exception as e:
            # Handle any exceptions that may occur during line wrapping
            print(f"Error wrapping lines: {e}")
            
    response_body = json.loads(response.get('body').read())
    llm_output_parser(response_body.get('completion'))
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

## FMs can also help you brainstorm for plot ideas.

Ask Claude for generate some plot ideas using the prompt below:

In [ ]:
prompt_data = """
Human: as an award wining director, give me some ideas about a plot-point in a romantic comedy involving two young professionals who just met by chance in the supermarket after loosing track of one another after college.
Assistant:
"""

In [ ]:
from io import StringIO
import sys
import textwrap

try:

    body = json.dumps({
      "prompt": prompt_data,
      "max_tokens_to_sample":256,
      "top_k":250,
      "stop_sequences":[], #define phrases that signal the model to conclude text generation.
      "temperature":0.5, #Temperature controls randomness; higher values increase diversity, lower values boost predictability.
      "top_p":1 # Top P is a text generation technique, sampling from the most probable tokens in a distribution.
    })

    response = bedrock_runtime.invoke_model(
      body=body,
      modelId="anthropic.claude-v2", 
      accept="application/json", 
      contentType="application/json"
    )


    def llm_output_parser(*args, width: int = 100, **kwargs):
        buffer = StringIO()
        try:
            # Redirect sys.stdout to capture the output
            _stdout = sys.stdout
            sys.stdout = buffer
            print(*args, **kwargs)
            output = buffer.getvalue()
        except Exception as e:
            # Handle any exceptions that may occur during capturing
            print(f"Error capturing output: {e}")
            return
        finally:
            # Restore the original sys.stdout
            sys.stdout = _stdout
    
        try:
            # Wrap lines and print the parsed output
            for line in output.splitlines():
                print("\n".join(textwrap.wrap(line, width=width)))
        except Exception as e:
            # Handle any exceptions that may occur during line wrapping
            print(f"Error wrapping lines: {e}")
            
    response_body = json.loads(response.get('body').read())
    llm_output_parser(response_body.get('completion'))
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

---
# Lab 3
# Image Generation in Bedrock

Now that you have text generation , let's try image generation with Titan Image Generator  on Bedrock.

>Before we get started lets restart the kernel. 

### Image Prompting

Writing a good prompt can be somewhat of an art.

It is often difficult to predict whether a given prompt will yield a satisfactory result with a certain model.

However, there are certain templates that have been known to work.

Broadly, a prompt can be broken down into three pieces:
- Type of image (photograph/sketch/painting)
- Description of the content (subject/object/environment/scene/&c.), and
- Style of the image (realistic/artistic).

You can change each of the three parts individually to generate variations of an image.
Adjectives have been known to play a significant role in the image generation process.
Also, adding more details help in the generation process.

In order to generate a realistic image, you can use phrases such as:
- a photo of
- a photograph of
- realistic
- hyper realistic

To generate something more artistic, you can use phrases like:
- by Pablo Picasso
- oil painting by Rembrandt
- landscape art by Frederic Edwin Church
- pencil drawing by Albrecht Dürer

You can also combine different artists as well.
To generate artistic images by category, you can add the art category in the prompt such as
lion on a beach, abstract

Some other categories include:
- oil painting
- pencil drawing
- pop art
- digital art
- anime
- cartoon
- futurism
- watercolor
- manga


You can also include details such as lighting or camera lens such as:
- 35mm wide lens
- 85mm wide lens

or details about the framing:
- portrait
- landscape
- close up

Note that models can generate different images even if same prompt is given multiple times.

So, you can generate multiple images and select the image that suits your application best.

For more information on Amazon Titan Image Generator prompt engineering, see [Amazon Titan Image Generator Prompt Engineering Best Practices.](https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Image+Generator+Prompt+Engineering+Guidelines.pdf)

In [ ]:
# Built-in libraries
import base64
import io
import json
import os
import sys

# External dependencies
import boto3
from PIL import Image
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')


## Text to Image

In text-to-image mode, we provide a text description (prompt) of the image that should be generated.

What if we want to avoid specific content or stylistic choices? Because image generation models are typically trained from image descriptions, trying to directly specify what you don't want in the prompt (e.g. man without a beard) doesn't usually work well: it would be very unusual to describe an image by what it is not!

In the case of Amazon Titan Image Generator, we can specify a negative prompt to steer the model away from unwanted elements

For our attempt we will use "a photograph of an astronaut riding a horse" and "nsfw"

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
negative_prompts = "nsfw"

The Amazon Bedrock InvokeModel provides access to Amazon Titan Image Generator by setting the right model ID, and returns a JSON response including a Base64 encoded string that represents the (PNG) image.

When making an InvokeModel request, we need to fill the body field with a JSON object that varies depending on the task (taskType) you wish to perform viz. text to image, image variation, inpainting or outpainting. The Amazon Titan models supports the following parameters:

- cfgscale - determines how much the final image reflects the prompt
- seed - a number used to initialize the generation, using the same seed with the same prompt + settings combination will produce the same results
- numberOfImages - the number of times the image is sampled and produced
- quality - determines the output image quality (standard or premium)

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,                    # Required
            "negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 1024,        # Supported height list in the docs 
            "width": 1024,         # Supported width list in the docs
            "cfgScale": 8,       # Range: 1.0 (exclusive) to 10.0
            "seed": 1             # Range: 0 to 214783647
        }
    }
)

# Make model request
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v1",
    accept="application/json", 
    contentType="application/json"
)

# Process the image
response_body = json.loads(response.get("body").read())
img1_b64 = response_body["images"][0]

# Debug
print(f"Output: {img1_b64[0:80]}...")

By decoding our image string and loading it with an image processing library like [Pillow](https://pillow.readthedocs.io/en/stable/), we can display and manipulate the image


In [ ]:
img1 = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img1_b64, "utf-8")
        )
    )
)
#print(os.getcwd())
img1.save("image_1.png")

# Display
img1

## Modify an Image

You can modify the image you just generated, this will further constrain the image generated. Let's change the prompt to **_Photograph of a astronaut riding a llama_**

In [ ]:
prompt = "a photograph of an astronaut riding a llama"
negative_prompts = "nsfw"

We will read the image file to a base64 object to pass to the model.

In [ ]:
def image_to_base64(img) -> str:
    """Converts a PIL Image or local image file path to a base64 string"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")

img1_b64 = image_to_base64(img1)
print(f"Input: {img1_b64[:80]}")

Lets pass this image to the model with our prompt.

In [ ]:
# Payload creation
body = json.dumps({
     "taskType": "IMAGE_VARIATION",
     "imageVariationParams": {
         "text": prompt,              # Optional
         "negativeText": negative_prompts,   # Optional
         "images": [img1_b64],               # One image is required
     },
     "imageGenerationConfig": {
         "numberOfImages": 1,
         "quality": "premium",
         "height": 1024,
         "width": 1024,
         "cfgScale": 8,
         "seed": 1
     }
 })

# Model invocation
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v1",
    accept="application/json", 
    contentType="application/json"
)

# Output processing
response_body = json.loads(response.get("body").read())
img2_b64 = response_body["images"][0]

# Debug
print(f"Output: {img2_b64[0:80]}...")

In [ ]:
img2 = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img2_b64, "utf-8")
        )
    )
)
#print(os.getcwd())
img2.save("image_2.png")

# Display
img2

### Inpainting

Another way to modify images is by using inpainting.

Inpainting refers to the process of replacing a portion of an image with another image based on a textual prompt.

Let's define what we want to change in the image.

In [ ]:
prompt = "a photograph of an astronaut riding a camel"

Next we will pass the previous image to the model with the mask.

In [ ]:
# Payload creation
body = json.dumps({
    "taskType": "INPAINTING",
    "inPaintingParams": {
        "text": prompt,              # Optional
        "negativeText": negative_prompts,    # Optional
        "image": img2_b64,      # Required
        "maskPrompt": "llama",               # One of "maskImage" or "maskPrompt" is required
        # "maskImage": image_to_base64(mask),  # Input maskImage based on the values 0 (black) or 255 (white) only
    },                                                 
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "premium",
        "height": 1024,
        "width": 1024,
        "cfgScale": 8,
        "seed": 0
    }
})

# Model invocation
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v1",
    accept="application/json", 
    contentType="application/json"
)

# Output processing
response_body = json.loads(response.get("body").read())
img3_b64 = response_body["images"][0]
print(f"Output: {img3_b64[0:80]}...")

In [ ]:
inpaint = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img3_b64, "utf-8")
        )
    )
)
inpaint.save("image_3.png")
inpaint

----
# Lab 4
# Question Answering in Bedrock

Question answering (QA) is an important task that involves extracting answers to factual queries posed in natural language. Foundation Models (FMs) like Amazon Titan, Anthropic Claude, and Cohere Command are trained on vast amounts of text. From that training, they develop the probability distributions that can predict next token/word in an answer sequence when given a question.

Despite how good these models have become, they are prone to **hallucination**, a phenomenon where a FMs generates false or inaccurate information. In the lab, we will experiment with different techniques to reduce hallucination and improve model response accuracy.

Lets ask some questions of the Amazon Titan Model:
- What is an Academy Award?
- In what year was the first Academy Awards ceremony?
- Who holds the record for the most Oscars won?
- Who nominates oscar nominees?

In [ ]:
import boto3
import botocore
import json 

bedrock_runtime = boto3.client('bedrock-runtime')

In [ ]:
prompt_data = """
You are a question and answer chatbot. Please answer the following question.

{QUESTION}
"""

In [ ]:
try:

    input_text = prompt_data.replace("{QUESTION}", "What is an Academy Award?")
    body = json.dumps({
      "inputText": input_text, 
      "textGenerationConfig":{  
        "maxTokenCount":128,
        "stopSequences":["User:"], #define phrases that signal the model to conclude text generation.
        "temperature":0, #Temperature controls randomness; higher values increase diversity, lower values boost predictability.
        "topP":0.9 # Top P is a text generation technique, sampling from the most probable tokens in a distribution.
      }
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="amazon.titan-text-express-v1",
      accept="application/json", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    outputText = response_body.get('results')[0].get('outputText')

    print(outputText)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

Change the prompt_data value above to try different questions.

Here are some examples that should work:
- What is an Academy Award?
- In what year was the first Academy Awards ceremony?
- Who holds the record for the most Oscars won?
- Who nominates oscar nominees?

So far so good. Now let's try a few question where the model may not answer correctly:
- What are the “Big Five” awards?
- In what year did ‘Forrest Gump’ win Best Picture?
- In what year was the first Oscar for Best Animated Feature awarded?

In [ ]:
try:

    input_text = prompt_data.replace("{QUESTION}", "What are the “Big Five” awards?")
    body = json.dumps({
      "inputText": input_text, 
      "textGenerationConfig":{  
        "maxTokenCount":128,
        "stopSequences":["User:"], #define phrases that signal the model to conclude text generation.
        "temperature":0, #Temperature controls randomness; higher values increase diversity, lower values boost predictability.
        "topP":0.9 # Top P is a text generation technique, sampling from the most probable tokens in a distribution.
      }
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="amazon.titan-text-express-v1",
      accept="application/json", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    outputText = response_body.get('results')[0].get('outputText')

    print(outputText)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

The model hallucinated, so let's try to fix it with prompt engineering.

In [ ]:
prompt_data = """
You are a question and answer chatbot. Please answer the following question. Say "I don't know" if you are not sure.

{QUESTION}
"""

In [ ]:
try:
    input_text = prompt_data.replace("{QUESTION}", "What are the “Big Five” awards?")

    body = json.dumps({
      "inputText": input_text, 
      "textGenerationConfig":{  
        "maxTokenCount":128,
        "stopSequences":["User:"], #define phrases that signal the model to conclude text generation.
        "temperature":0, #Temperature controls randomness; higher values increase diversity, lower values boost predictability.
        "topP":0.9 # Top P is a text generation technique, sampling from the most probable tokens in a distribution.
      }
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="amazon.titan-text-express-v1",
      accept="application/json", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    outputText = response_body.get('results')[0].get('outputText')

    print(outputText)

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

## In-Context Learning
Now let's help the model answer this correctly.

Following passage is copied from Wikipedia: [List of Big Five Academy Award winners and nominees.](https://en.wikipedia.org/wiki/List_of_Big_Five_Academy_Award_winners_and_nominees) Let's also feed this passage to Titan as context.

>At the Academy Awards, the so-called "Big Five" awards are those for Best Picture, Best Director, Best Actor, Best Actress, and Best Screenplay (either Best Original Screenplay or Best Adapted Screenplay).[1] As of the 94th Academy Awards (2021), a total of 43 films have been nominated in all five of these award categories. Only three films have won all five of these major awards: It Happened One Night (1934), One Flew Over the Cuckoo's Nest (1975), and The Silence of the Lambs (1991). Eight films failed to win any of the five major awards after being nominated.

In [ ]:
context = """
At the Academy Awards, the so-called "Big Five" awards are those for Best Picture, 
Best Director, Best Actor, Best Actress, and Best Screenplay (either Best Original 
Screenplay or Best Adapted Screenplay).[1] As of the 94th Academy Awards (2021), a 
total of 43 films have been nominated in all five of these award categories. Only 
three films have won all five of these major awards: It Happened One Night (1934), 
One Flew Over the Cuckoo's Nest (1975), and The Silence of the Lambs (1991). Eight 
films failed to win any of the five major awards after being nominated.
"""

In [ ]:
prompt_data = """
You are a question and answer chatbot. Please answer the following question only use the context below. Say "I don't know" if you are not sure.

{CONTEXT}

{QUESTION}
"""

In [ ]:
try:

    input_text = prompt_data.replace("{QUESTION}", "What are the “Big Five” awards?")
    input_text = input_text.replace("{CONTEXT}", context)


    body = json.dumps({
      "inputText": input_text, 
      "textGenerationConfig":{  
        "maxTokenCount":128,
        "stopSequences":[], #define phrases that signal the model to conclude text generation.
        "temperature":0, #Temperature controls randomness; higher values increase diversity, lower values boost predictability.
        "topP":0.9 # Top P is a text generation technique, sampling from the most probable tokens in a distribution.
      }
    })

    response = bedrock_runtime.invoke_model(
      body=body,
	  modelId="amazon.titan-text-express-v1",
      accept="application/json", 
      contentType="application/json"
    )

    response_body = json.loads(response.get('body').read())
    outputText = response_body.get('results')[0].get('outputText')

    print(outputText)
    
except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

>Thank you, you have completed this section of the workshop.